This notebook gives an example function to call the watsonx.ai text generation API.

Models available on watsonx, with their model IDs: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx  
API Documentation: https://cloud.ibm.com/apidocs/watsonx-ai#text-generation

In [1]:
import requests
import json

In [2]:
def get_token(apikey):
    token_resp = requests.post("https://iam.bluemix.net/identity/token",
                                headers={
                                     "Content-Type": "application/x-www-form-urlencoded",
                                     "Accept":"application/json"
                                     },
                                data={
                                     "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
                                     "apikey": apikey
                                 },
                                verify=True)
    
    token = json.loads(token_resp.content.decode("utf-8")).get("access_token")
    return token

def prompt_watsonx_model(model_id, project_id, prompt, token):
    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
    
    body = {
    	"input": prompt,
    	"parameters": {
    		"decoding_method": "greedy",
    		"max_new_tokens": 2000,
    		"min_new_tokens": 0,
    		"repetition_penalty": 1
    	},
    	"model_id": model_id,
    	"project_id": project_id,
    	
    }
    
    headers = {
    	"Accept": "application/json",
    	"Content-Type": "application/json",
    	"Authorization": f"Bearer {token}"
    }
    
    response = requests.post(
    	url,
    	headers=headers,
    	json=body
    )
    
    if response.status_code != 200:
    	raise Exception("Non-200 response: " + str(response.text))
    
    data = response.json()
    return data["results"][0]["generated_text"]

In [3]:
# itz-watsonx-6 account api key
APIKEY= "api_key"
project_id= "project_id"

model_id= "meta-llama/llama-4-maverick-17b-128e-instruct-fp8"

prompt= """
<|begin_of_text|><|header_start|>system<|header_end|>

You are a helpful assistant<|eot|><|header_start|>user<|header_end|>

Write 3 question and answer pairs related to the FFIEC guidelines<|eot|><|header_start|>assistant<|header_end|>"""

token= get_token(APIKEY)
output= prompt_watsonx_model(model_id, project_id, prompt, token)

In [4]:
print(output)



Here are 3 question and answer pairs related to the FFIEC guidelines:

**Q1: What is the purpose of the FFIEC guidelines?**
A1: The purpose of the FFIEC (Federal Financial Institutions Examination Council) guidelines is to provide a framework for financial institutions to manage risk and ensure the security and privacy of customer information. The guidelines provide guidance on various aspects of information security, including authentication, risk management, and incident response.

**Q2: What are the key elements of the FFIEC's Authentication Guidance?**
A2: The FFIEC's Authentication Guidance emphasizes the need for financial institutions to implement robust authentication controls, including multi-factor authentication, to prevent unauthorized access to customer accounts and sensitive information. The guidance identifies four key elements: (1) understanding the risk, (2) implementing effective authentication mechanisms, (3) monitoring and responding to authentication-related inci